$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}$

# Assignment 6: Neural Networks

Jared Zymbaluk

In [30]:
import numpy as np
np.set_printoptions(8)
import matplotlib.pyplot as plt
import mlutils as ml
import neuralnetworks as nn
import scaledconjugategradient as scg
import time
import math
import copy

In [2]:
def trainNNs(X, T, trainFraction, hiddenLayerStructures, numberRepetitions, numberIterations, classify=False):
    collection = [] 
    for structure in hiddenLayerStructures:
        data = []
        trained_data = []
        test_data = []
        start = time.time()
        for num in range(numberRepetitions):
            if classify:
                Xtrain,Ttrain,Xtest,Ttest = ml.partition(X,T,(trainFraction,1-trainFraction),classify)
            else:
                Xtrain,Ttrain,Xtest,Ttest = ml.partition(X,T,(trainFraction,1-trainFraction))
            network = nn.NeuralNetwork(X.shape[1], structure, T.shape[1])
            network.train(Xtrain, Ttrain, numberIterations)
            Y = network.use(Xtrain)
            Ytest = network.use(Xtest)
            
            if classify:
                
                trained_data.append(np.sum(Y.ravel()==Ttrain.ravel()) / float(len(Ttrain)) * 100)
                test_data.append(np.sum(Ytest.ravel()==Ttest.ravel()) / float(len(Ttest)) * 100)
            else:
                trained_data.append(np.sqrt(np.mean(Y-Ttrain) ** 2))
                test_data.append(np.sqrt(np.mean(Ytest-Ttest) ** 2))
        end = time.time()
        listy = [structure,trained_data,test_data,end-start]
        collection.append(listy)
        
            
    return collection
    

In [3]:
def summarize(results):
    copyres = copy.deepcopy(results)
    for i in copyres:
        for res in range(len(i)):
            if res == 0:
                continue
            elif res == len(i)-1:
                continue
            else:
                i[res] = (sum(i[res])/len(i[res]))
             
             #print(i)
    return copyres
    

In [4]:
def bestNetwork(summary):
    best = summary[0]
    for i in summary:
        if i[2] < best[2]:
            best = i
    return best

## Data for Regression Experiment

From the UCI Machine Learning Repository, download the [Appliances energy prediction](http://archive.ics.uci.edu/ml/datasets/Appliances+energy+prediction) data.  You can do this by visiting the Data Folder for this data set, or just do this:

     !wget http://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv



In [5]:
file = np.genfromtxt('energydata_complete.csv', dtype='str',delimiter=',',deletechars='"')
file = np.char.replace(file, '"', '')
file = np.char.replace(file, ' ', '')
file = np.delete(file, 0,1)
file = np.delete(file, -1,1)
file = np.delete(file, -1,1)
names = file[0]
names = names.astype(np.str)
data = file[1:]
data = data.astype(np.float)
#file.chararray.strip('"')

Read this data into python.  One suggestion is to use the `loadtxt` function in the `numpy` package.  You may ignore the first column of each row which contains a data and time.  Also ignore the last two columns of random variables.  We will not use that in our modeling of this data.  You will also have to deal with the double quotes that surround every value in every field.  Read the first line of this file to get the names of the features.

Once you have read this in correctly, you should see values like this:

In [6]:
names

array(['Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3',
       'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint'], 
      dtype='<U23')

In [7]:
data.shape

(19735, 26)

In [8]:
data[:2,:]

array([[  60.        ,   30.        ,   19.89      ,   47.59666667,
          19.2       ,   44.79      ,   19.79      ,   44.73      ,
          19.        ,   45.56666667,   17.16666667,   55.2       ,
           7.02666667,   84.25666667,   17.2       ,   41.62666667,
          18.2       ,   48.9       ,   17.03333333,   45.53      ,
           6.6       ,  733.5       ,   92.        ,    7.        ,
          63.        ,    5.3       ],
       [  60.        ,   30.        ,   19.89      ,   46.69333333,
          19.2       ,   44.7225    ,   19.79      ,   44.79      ,
          19.        ,   45.9925    ,   17.16666667,   55.2       ,
           6.83333333,   84.06333333,   17.2       ,   41.56      ,
          18.2       ,   48.86333333,   17.06666667,   45.56      ,
           6.48333333,  733.6       ,   92.        ,    6.66666667,
          59.16666667,    5.2       ]])

Use the first two columns, labelled `Appliances` and `lights` as the target variables, and the remaining 24 columns as the input features.  So

In [9]:
Tenergy = np.take(data,[0,1],1)
Xenergy = np.take(data,range(2,26),1)
Tnames = np.take(names,[0,1])
Xnames = np.take(names,range(2,26))

In [10]:
Xenergy.shape, Tenergy.shape

((19735, 24), (19735, 2))

In [11]:
Xnames

array(['T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5',
       'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9',
       'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility',
       'Tdewpoint'], 
      dtype='<U23')

In [12]:
Tnames

array(['Appliances', 'lights'], 
      dtype='<U23')

Train several neural networks on all of this data for 100 iterations.  Plot the error trace (nnet.getErrorTrace()) to help you decide now many iterations might be needed.  100 may not be enough.  If for your larger networks the error is still decreasing after 100 iterations you should train all nets for more than 100 iterations.

Now use your `trainNNs`, `summarize`, and `bestNetwork` functions on this data to investigate various network sizes.

In [13]:
nnet1 = nn.NeuralNetwork(Xenergy.shape[1], 0, Tenergy.shape[1])
nnet1.train(Xenergy, Tenergy, 100)
plt.plot(nnet1.getErrorTrace())

In [14]:
nnet2 = nn.NeuralNetwork(Xenergy.shape[1], [5,5], Tenergy.shape[1])
nnet2.train(Xenergy, Tenergy, 500)
plt.plot(nnet2.getErrorTrace())

In [15]:
nnet3 = nn.NeuralNetwork(Xenergy.shape[1], [10,10,10], Tenergy.shape[1])
nnet3.train(Xenergy, Tenergy, 1000)
plt.plot(nnet3.getErrorTrace())

In [ ]:
results = trainNNs(Xenergy, Tenergy, 0.8, [0, 5, [5, 5],10,[10,10,10],[1,1],[1,2,3],[6,6,6,6], [10, 10],1], 10, 100)

In [ ]:
summarize(results)

In [ ]:
bestNetwork(summarize(results))

Test at least 10 different hidden layer structures.  Larger numbers of layers and units may do the best on training data, but not on testing data. Why?

Now train another network with your best hidden layer structure on 0.8 of the data and use the trained network on the testing data (the remaining 0.2 of the date).  As before use `ml.partition` to produce the training and testing sets.

For the testing data, plot the predicted and actual `Appliances` energy use, and the predicted and actual `lights` energy use, in two separate plots.  Discuss what you see.

In [16]:
Xtrain,Ttrain,Xtest,Ttest = ml.partition(Xenergy,Tenergy,(0.8,1-0.8))
network3 = nn.NeuralNetwork(Xenergy.shape[1], 10, Tenergy.shape[1])
network3.train(Xenergy, Tenergy, 100)
Y = network3.use(Xtrain)
Ytest = network3.use(Xtest)

In [20]:
plt.plot(np.take(Y,0,1))

In [21]:
plt.plot(np.take(Ytest,0,1))

I see that the actual energy used is very close to the predicted energy used. Impressive!

## Data for Classification Experiment

From the UCI Machine Learning Repository, download the [Anuran Calls (MFCCs)](http://archive.ics.uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29) data.  You can do this by visiting the Data Folder for this data set, or just do this:

     !wget 'http://archive.ics.uci.edu/ml/machine-learning-databases/00406/Anuran Calls (MFCCs).zip'
     !unzip Anuran*zip
     
Read the data in the file `Frogs_MFCCs.csv` into python.  This will be a little tricky. Each line of the file is a sample of audio features plus three columns that label the sample by family, genus, and species. We will try to predict the species.  The tricky part is that the species is given as text.  We need to convert this to a target class, as an integer. The `numpy` function `unique` will come in handy here.

In [23]:
file2=""
file2 = np.loadtxt('Frogs_MFCCs.csv', dtype='str',delimiter=',',skiprows=1)
file2 = np.delete(file2, -1,1)
file2 = np.delete(file2, -2,1)
file2 = np.delete(file2, -2,1)
Tanuran = np.take(file2, -1,1)
test = np.unique(Tanuran)
i = 0
for string in test:
    Tanuran[Tanuran==string]=[i]
        
            
    i=i+1
Tanuran = Tanuran.astype(int)
Tanuran = Tanuran.reshape((-1,1))
Xanuran = np.delete(file2,-1,1)
Xanuran = np.char.strip(Xanuran, "b'")
Xanuran = Xanuran.astype(float)

In [24]:
Xanuran.shape, Tanuran.shape

((7195, 22), (7195, 1))

In [25]:
Xanuran[:2,:]

array([[ 1.        ,  0.1529363 , -0.1055859 ,  0.20072191,  0.31720106,
         0.26076385,  0.10094464, -0.1500626 , -0.17112763,  0.12467644,
         0.18865415, -0.07562172, -0.15643593,  0.08224512,  0.13575204,
        -0.02401665, -0.10835111, -0.07762252, -0.0095678 ,  0.05768398,
         0.11868014,  0.01403845],
       [ 1.        ,  0.17153426, -0.09897474,  0.26842522,  0.33867186,
         0.2683531 ,  0.06083509, -0.22247464, -0.20769267,  0.17088287,
         0.27095828, -0.09500394, -0.25434147,  0.02278623,  0.1633201 ,
         0.01202228, -0.09097401, -0.05650952, -0.03530336,  0.02013996,
         0.08226299,  0.02905574]])

In [26]:
Tanuran[:2]

array([[0],
       [0]])

In [27]:
for i in range(10):
    print('{} samples in class {}'.format(np.sum(Tanuran==i), i))

672 samples in class 0
3478 samples in class 1
542 samples in class 2
310 samples in class 3
472 samples in class 4
1121 samples in class 5
270 samples in class 6
114 samples in class 7
68 samples in class 8
148 samples in class 9


In [28]:
results = trainNNs(Xanuran, Tanuran, 0.8, [0, 5, [5, 5]], 5, 100, classify=True)

In [31]:
summarize(results)

[[0, 0.0, 0.0, 0.14337491989135742],
 [5, 0.0, 0.0, 2.3392398357391357],
 [[5, 5], 0.0, 0.0, 3.0005013942718506]]

In [32]:
bestNetwork(summarize(results))

[0, 0.0, 0.0, 0.14337491989135742]

Now do an investigation like you did for the regression data. 

Test at least 10 different hidden layer structures. Then train another network with your best hidden layer structure on 0.8 of the data and use the trained network on the testing data (the remaining 0.2 of the date). 

Plot the predicted and actual `Species` for the testing data as an integer.  Discuss what you see.

In [33]:
results2 = trainNNs(Xanuran, Tanuran, 0.8, [0,1,2,3,4,5,[1,1], [2,2,2],[2,2],[4,2,3,4]], 5, 100, classify=True)

In [34]:
summarize(results2)

[[0, 0.0, 0.0, 0.22059178352355957],
 [1, 0.0, 0.0, 1.491978406906128],
 [2, 0.0, 0.0, 1.7957885265350342],
 [3, 0.0, 0.0, 2.1432151794433594],
 [4, 0.0, 0.0, 2.445521354675293],
 [5, 0.0, 0.0, 2.3988966941833496],
 [[1, 1], 0.0, 0.0, 1.8444180488586426],
 [[2, 2, 2], 0.0, 0.0, 2.85010027885437],
 [[2, 2], 0.0, 0.0, 1.967747449874878],
 [[4, 2, 3, 4], 0.0, 0.0, 3.802138328552246]]

In [35]:
bestNetwork(summarize(results2))

[0, 0.0, 0.0, 0.22059178352355957]

In [36]:
Xtrain,Ttrain,Xtest,Ttest = ml.partition(Xanuran,Tanuran,(0.8,1-0.8),True)
network4 = nn.NeuralNetworkClassifier(Xanuran.shape[1], 0, 10,)
network4.train(Xanuran, Tanuran, 100)
Y = network3.use(Ttrain)
Ytest = network3.use(Ttest)

In [37]:
plt.plot(np.take(Y,0,1))

In [38]:
plt.plot(np.take(Ytest,0,1))

^The predicted values match up very closely with the test values!